In [1]:
import numpy as np
import torch
import pandas as pd
import torchvision
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm.notebook import tqdm
from typing import Optional
import sys

/home/sasha/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
sys.path.insert(0, "../src")
print(sys.path)

['../src', '/home/sasha/BMM/bayes_deep_compression/examples', '/home/sasha/anaconda3/lib/python311.zip', '/home/sasha/anaconda3/lib/python3.11', '/home/sasha/anaconda3/lib/python3.11/lib-dynload', '', '/home/sasha/anaconda3/lib/python3.11/site-packages']


In [3]:

class Classifier(nn.Module): 
    def __init__(self, classes: int = 10): 
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) 
        self.pool = nn.MaxPool2d(2, 2) 
        #self.dropout1 = nn.Dropout2d(0.25) 
        #self.dropout2 = nn.Dropout2d(0.5) 
        self.fc1 = nn.Linear(64 * 7 * 7, 128) 
        self.fc2 = nn.Linear(128, classes) 
  
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x))) 
        #x = self.dropout1(x) 
        x = self.pool(F.relu(self.conv2(x))) 
        #x = self.dropout2(x) 
        x = x.view(-1, 64 * 7 * 7) 
        x = F.relu(self.fc1(x)) 
        x = self.fc2(x) 
        return x

In [4]:
from bayescomp.bayes.variational.net import VarBayesModuleNet
from bayescomp.bayes.variational.net import NormalVarBayesModule
from bayescomp.bayes.variational.optimization_renui import VarRenuiLoss

In [5]:
from bayescomp.bayes.variational.trainer import VarBayesTrainer, VarTrainerParams, Beta_Scheduler_Plato, CallbackLossAccuracy
from bayescomp.bayes.variational.trainer import Beta_Scheduler
from bayescomp.report.base import ReportChain
from bayescomp.report.variational import VarBaseReport

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
test_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [8]:
BATCH_SIZE=512
EPOCHS=11
LR = 5e-4 #5e-4
# Split the training set into training and validation sets 
VAL_PERCENT = 0.2 # percentage of the data used for validation 
SAMPLES = 10
BETA = 0.01 #5e-5 #len(train_dataset) *1. / BATCH_SIZE
BETA_FAC = 5e-1
PRUNE = -1.#1.99, 2.1
PLATO_TOL = 20

base_module = Classifier()
var_module = NormalVarBayesModule(base_module)
model = VarBayesModuleNet(base_module, nn.ModuleList([var_module]))


optimizer = optim.Adam(model.parameters(), lr=LR)

fit_loss = nn.CrossEntropyLoss(reduction="sum") 
kl_loss = VarRenuiLoss()
beta = len(train_dataset) *1. / BATCH_SIZE
print(beta)
#beta = Beta_Scheduler(beta=(len(train_dataset) *1. / BATCH_SIZE))

# beta_KL = Beta_Scheduler_Plato(beta.beta, 1 / BETA_FAC, PLATO_TOL, ref = beta, threshold=1e-4)

#Данная функция будет выполнятся после каждого шага тренера, соответсвенно нам требуется сделать шаг планировщика и изменить соотвествующий коэффициент
    
#print(model.base_module.state_dict().keys())
val_size    = int(VAL_PERCENT * len(train_dataset)) 
train_size  = len(train_dataset) - val_size 

t_dataset, v_dataset = torch.utils.data.random_split(train_dataset,  
                                                        [train_size,  
                                                            val_size]) 

# Create DataLoaders for the training and validation sets 
train_loader = torch.utils.data.DataLoader(t_dataset,  
                                        batch_size=BATCH_SIZE,  
                                        shuffle=True, 
                                        pin_memory=True) 

eval_loader = torch.utils.data.DataLoader(v_dataset,  
                                        batch_size=BATCH_SIZE,  
                                        shuffle=False, 
                                        pin_memory=True)

model.to(device) 
train_params = VarTrainerParams(EPOCHS, optimizer,fit_loss, kl_loss, SAMPLES, PRUNE, beta, {'accuracy': CallbackLossAccuracy()})
#trainer = VarBayesTrainer(train_params, ReportChain([VarBaseReport()]), train_loader, eval_loader, [post_train_step])
trainer = VarBayesTrainer(train_params, ReportChain([VarBaseReport()]), train_loader, eval_loader)
trainer.train(model)

19.53125


  0%|          | 0/11 [00:00<?, ?it/s]

Epoch [1/11],Loss:403944352.0, KL Loss: 41035676.0. FitLoss: 30119552.0,Accuracy:0.10039550781249999,Validation Loss:182780688.0,Validation Accuracy:0.14439655172413793, Prune parameters: 270484.0/421642,Beta: 19.53125
Epoch [2/11],Loss:373721472.0, KL Loss: 41007604.0. FitLoss: 27884230.0,Accuracy:0.10023437500000001,Validation Loss:161244624.0,Validation Accuracy:0.1400862068965517, Prune parameters: 270621.0/421642,Beta: 19.53125
Epoch [3/11],Loss:350995680.0, KL Loss: 41020492.0. FitLoss: 25929834.0,Accuracy:0.1011083984375,Validation Loss:134965856.0,Validation Accuracy:0.125, Prune parameters: 270698.0/421642,Beta: 19.53125
Epoch [4/11],Loss:354312672.0, KL Loss: 41004048.0. FitLoss: 24591042.0,Accuracy:0.10059570312500002,Validation Loss:121780464.0,Validation Accuracy:0.1271551724137931, Prune parameters: 270813.0/421642,Beta: 19.53125
Epoch [5/11],Loss:319801536.0, KL Loss: 41014984.0. FitLoss: 22887788.0,Accuracy:0.10638427734375,Validation Loss:109702496.0,Validation Accurac

KeyboardInterrupt: 